In [8]:
import tkinter as tk
from tkinter import ttk
from tkinter import * 

import cv2
from tensorflow import keras
from tensorflow.keras import layers
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

import winsound
frequency = 2500
duration = 1000
import numpy as np

path = "haarcascade_fronalface_default.xml"
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_frontalface_alt.xml')
detector = False
counter = 0
font = cv2.FONT_HERSHEY_SIMPLEX


new_model = keras.models.load_model('new_model.h5')

# this is the function called when the button is clicked
def btnClickFunction():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        raise IOError("Cannot open webcam")

    while True:
        ret,frame = cap.read()
        eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_eye_tree_eyeglasses.xml')
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        eyes = eye_cascade.detectMultiScale(gray,1.1,4)
        for x,y,w,h in eyes:
            roi_gray = gray[y:y+h , x:x+w]
            roi_color = frame[y:y+h , x:x+w]
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2 )
            eyess = eye_cascade.detectMultiScale(roi_gray)
            if len(eyess) == 0:
                print ("eyes are not detected")
            else:
                for (ex,ey,ew,eh) in eyess:
                    eyes_roi = roi_color[ey: ey+eh, ex:ex + ew]
        #Here was the error , it's because you write 244 instead of 224 , because pretrained model use 224pixel * 224pixel images
        #for training so we have to use the same size for validation images and also must be RGB
        final_image =cv2.resize(eyes_roi, (224,224))
        final_image = np.expand_dims(final_image,axis =0)
        final_image = final_image/255.0;
        Predictions = new_model.predict(final_image)
        print(str(Predictions))
        if(Predictions >=  0.1):
            detector = True
            for x,y,w,h in eyes:
                roi_gray = gray[y:y+h , x:x+w]
                roi_color = frame[y:y+h , x:x+w]
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2 )
                eyess = eye_cascade.detectMultiScale(roi_gray)
                for (ex,ey,ew,eh) in eyess:
                    eyes_roi = roi_color[ey: ey+eh, ex:ex + ew]
        #Here was the error , it's because you write 244 instead of 224 , because pretrained model use 224pixel * 224pixel images
        #for training so we have to use the same size for validation images and also must be RGB
                x1,y1,w1,h1 = 0,0,175,75
                cv2.rectangle(frame, (x1, x1), (x1+w1, y1+h1), (0,0,0),-1)
                cv2.putText(frame,"Open eyes" , (x1 + int(w1/10),y1+int(h1/2)),font, 0.7, (0,255,0),2)
        else:
            if detector == False:
                counter = counter + 1
                x1,y1,w1,h1 = 0,0,175,75
                cv2.rectangle(frame, (x1, x1), (x1+w1, y1+h1), (0,0,0),-1)
                cv2.putText(frame,"Closed Eyes" , (x1 + int(w1/10),y1+int(h1/2)),font, 0.7, (0,0,255),2)
                if counter>5:
                    x1,y1,w1,h1 = 0,0,175,75
                    cv2.rectangle(frame, (x1, x1), (x1+w1, y1+h1), (0,0,0),-1)
                    cv2.putText(frame, "Wake up !!", (x1 + int(w1/10),y1+int(h1/2)),font, 0.7, (0,0,255),2)
                    winsound.Beep(frequency, duration)
                    counter = 0
            detector = False
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        print(faceCascade.empty())
        faces = faceCascade.detectMultiScale(gray,1.1,4)
        #Draw a rectangle around the face


        for(x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255,0,0), 2)

        # Use putText() method for
        # inserting text on video
        cv2.putText(frame,
               str(Predictions),
               (50, 50),
               font, 3,
               (0, 0, 255),
               2,
               cv2.LINE_4)
        print(type(Predictions))
        cv2.imshow('Drowsiness Detection Tutorial', frame)

        if cv2.waitKey(2) & 0xFF == ord('q'):
            break 

    cap.release()
    cv2.destroyAllWindows


# this is the function called when the button is clicked
#def btnClickFunction():
#	print('clicked')
    
# this is a function to get the user input from the text input box
def getInputBoxValue():
	userInput = Numero.get()
	return userInput


root = Tk()

# This is the section of code which creates the main window
root.geometry('668x436')
root.configure(background='#F0F8FF')
root.title('Hypovigilance Detection Application')

# This is the section of code which creates the a label
Label(root, text='Bienvenue sur', bg='#F0F8FF', font=('arial', 30, 'normal')).place(x=200, y=5)


# This is the section of code which creates the a label
Label(root, text='L\'Application à l\'Hypovigilance', bg='#F0F8FF', font=('arial', 30, 'normal')).place(x=95, y=45)


# This is the section of code which creates the a label
Label(root, text='Chez les Conducteurs', bg='#F0F8FF', font=('arial', 30, 'normal')).place(x=142, y=93)

# First, we create a canvas to put the picture on
photoo= Canvas(root, height=145, width=250)
# Then, we actually create the image file to use (it has to be a *.gif)
picture_file = PhotoImage(file = 'photo.gif')  # <-- you will have to copy-paste the filepath here, for example 'C:\Desktop\pic.gif'
# Finally, we create the image on the canvas and then place it onto the main window
photoo.create_image(250, 0, anchor=NE, image=picture_file)
photoo.place(x=220, y=146)


# This is the section of code which creates a button
Button(root, text='Commencer', bg='#00FF7F', font=('arial', 20, 'bold'), command=btnClickFunction).place(x=8, y=291)


# This is the section of code which creates a button
Button(root, text='Arrêter', bg='#B22222', font=('arial', 20, 'bold'), command=btnClickFunction).place(x=499, y=292)

# This is the section of code which creates the a label
Label(root, text='En Cas D\'urgence : ', bg='#F0F8FF', font=('arial', 15, 'bold')).place(x=7, y=365)

# This is the section of code which creates the a label
Label(root, text='Saisir le numéro à appeler :', bg='#F0F8FF', font=('arial', 18, 'bold')).place(x=86, y=395)


# This is the section of code which creates a text input box
Numero=Entry(root)
Numero.place(x=446, y=405)


root.mainloop()


[[1.2277786e-10]]


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\ASUS\anaconda3\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-8-dd57e49663ca>", line 71, in btnClickFunction
    if detector == False:
UnboundLocalError: local variable 'detector' referenced before assignment


[[3.285177e-06]]


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\ASUS\anaconda3\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-8-dd57e49663ca>", line 71, in btnClickFunction
    if detector == False:
UnboundLocalError: local variable 'detector' referenced before assignment


In [9]:
cap = cv2.VideoCapture(0)